In [79]:
import re
from typing import List, Tuple
from urllib.parse import urljoin
import json
from pathlib import Path

import requests
from bs4 import BeautifulSoup, NavigableString
from langchain.agents import AgentType, initialize_agent, create_react_agent
from langchain.chat_models import ChatOpenAI

from langchain.schema import HumanMessage, SystemMessage
from langchain.tools import Tool
from langchain_core.tools import tool

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from dotenv import load_dotenv
from typing import Annotated, List


import os

load_dotenv()


True

In [3]:
# Prompt sin usar
system_prompt = '''Assistant es un gran modelo de lenguaje entrenado por OpenAI.

Assistant está diseñado para ayudar en una amplia gama de tareas, desde responder a preguntas sencillas hasta proporcionar explicaciones detalladas y debates sobre una gran variedad de temas. Como modelo lingüístico, Assistant es capaz de generar textos similares a los humanos a partir de la información que recibe, lo que le permite entablar conversaciones naturales y ofrecer respuestas coherentes y relevantes para el tema en cuestión.

Assistant aprende y mejora constantemente, y sus capacidades evolucionan sin cesar. Es capaz de procesar y comprender grandes cantidades de texto, y puede utilizar este conocimiento para proporcionar respuestas precisas e informativas a una amplia gama de preguntas.             Además, Assistant es capaz de generar su propio texto a partir de la información que recibe, lo que le permite participar en debates y ofrecer explicaciones y descripciones sobre una amplia gama de temas.

En general, Assistant es un potente sistema que puede ayudarte con una gran variedad de tareas y proporcionarte valiosos conocimientos e información sobre una amplia gama de temas. Tanto si necesitas ayuda con una pregunta concreta como si sólo quieres mantener una conversación sobre un tema en particular, Assistant está aquí para ayudarte.'''


human_prompt = '''HERRAMIENTAS
------
Assistant puede pedir al usuario que utilice herramientas para buscar información que pueda ser útil para responder a la pregunta original del usuario. Las herramientas que el humano puede utilizar son:

{tools}

INSTRUCCIONES DE FORMATO DE RESPUESTA
----------------------------

Cuando me responda, por favor envíe una respuesta en uno de estos dos formatos:

**Opción 1:**
Utilice esta opción si desea que el humano utilice una herramienta.
Fragmento de código Markdown formateado en el siguiente esquema:

```json
{{
    "action": string, \ La acción a tomar. Debe ser uno de {tool_names}.
    "action_input": cadena \ La entrada de la acción
}}
```

**Opción #2:**
Utilice esta opción si desea responder directamente al humano. Markdown fragmento de código formateado en el siguiente esquema:

```json
{{
    "action": "Respuesta final",
    "action_input": string \ Usted debe poner lo que desea volver a utilizar aquí
}}
```

ENTRADA DEL USUARIO
--------------------
Aquí está el input del usuario (recuerda responder con un fragmento de código markdown de un blob json con una única acción, y NADA más):

{input}'''

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", human_prompt),
        MessagesPlaceholder("agent_scratchpad"),
    ]
)

In [54]:
# También sin uso
json_schema ={
  "title": "Faq",
  "description": "Lista de objetos JSON de pares 'question' y 'answer'",
  "type": "object",
  "properties": {
    "faq": {
      "type": "array",
      "description": "Lista de objetos JSON de pares 'question' y 'answer'",
      "items": {
        "type": "object",
        "properties": {
          "question": {
            "type": "string",
            "description": "Pregunta inferida del texto"
          },
          "answer": {
            "type": "string",
            "description": "Contenido del texto que responde a una pregunta"
          }
        },
        "required": ["question", "answer"]
      }
    }
  },
  "required": ["faq"]
}


In [120]:
# @tool("Agrupar texto en formato pregunta-respuesta")
def group_text(text: str) -> List[dict]:
    """
    Agrupa texto en una lista de objetos JSON con formato question-answer
    """
    llm = ChatOpenAI(temperature=0.1, model="gpt-4o-mini", openai_api_key=os.environ["OPENAI_API_KEY"])
    # llm = llm.bind_structured_output(json_schema)
    prompt = ChatPromptTemplate.from_messages([("system", "Del siguiente texto, para cada punto escribe una pregunta respondida por el texto. Luego entrega una lista de objetos JSON con atributos 'question' y 'answer'. Donde 'question' es la pregunta y 'answer' es la respuesta tomada tal como aparece en el texto. A continuación, el texto (recuerda responder con un fragmento de código markdown de un blob json con una única acción, y NADA más): \n\n {text}")])
    chain = prompt | llm
    grouped_text = chain.invoke({"text": text})
    parsed_qas = "".join(grouped_text.content.split('\n')[1:-1])
    # print(parsed_qas)
    question_answer_list = json.loads(parsed_qas)
    return question_answer_list


In [244]:
class FAQScraper:
    def __init__(self, base_url: str, model: str, openai_api_key: str):
        self.base_url = base_url
        self.visited_urls = set()
        self.results = []
        self.openai_api_key = openai_api_key

        # Initialize LangChain agent
        self.model = model
        self.llm = ChatOpenAI(temperature=0, model=self.model, openai_api_key=openai_api_key)
        tools = [self._group_text]
        self.agent = create_react_agent(self.llm, tools, prompt=prompt)

    def scrape(self, start_url: str, max_depth: int = 2) -> List[dict]:
        self._scrape_recursive(start_url, max_depth)
        return self.results
    
    def scrape_page(self, url: str) -> str:
        response = requests.get(url, timeout=10)  # Add a 30-second timeout
        soup = BeautifulSoup(response.text, 'html.parser')
        text_content = soup.get_text(separator='\n', strip=True)
        topic = text_content.split('\n')[0].split('|')[0].strip()
        # relevant_text = self.agent.run(f"Extrae del siguiente texto el contenido relacionado con el tema específico '{topic}', luego agrupa el contenido en un JSON en formato pregunta-respuesta:   \n\n{text_content}")
        grouped_text = group_text(text_content)
        # grouped_text = relevant_text
        return grouped_text
    
    def _extract_relevant_text(self, text_content: str, topic: str) -> str:
        relevant_text = self.llm.invoke(f"Extrae del siguiente texto el contenido relacionado con el tema específico '{topic}', solamente entrega el contenido relevante  \n\n{text_content}").content
        return relevant_text
    

    def _scrape_recursive(self, url: str, max_depth: int = 2, current_depth: int = 0):
        if url in self.visited_urls:
            return

        self.visited_urls.add(url)
        print(f"Scraping URL: {url}")
        response = requests.get(url, timeout=10)  # Add a 30-second timeout
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract text content
        text_content = soup.get_text(separator='\n', strip=True)
        topic = text_content.split('\n')[0].split('|')[0].strip()

        # Use LangChain agent to group text
        relevant_text = self.llm.invoke(f"Extrae del siguiente texto el contenido relacionado con el siguiente tema: {topic}, solamente entrega el texto  \n\n{text_content}").content
        question_answer_list = []
        try:
            question_answer_list = self._group_text(relevant_text)
        except Exception as e:
            try:
                question_answer_list = self._group_text(relevant_text)
            except Exception as e:
                print(f"Error grouping text from url {url}: {e}")
        
        for qa in question_answer_list:
            qa["category"] = topic
        
        self.results.extend(question_answer_list)

        # Find and follow links
        if current_depth >= max_depth:
            return
        
        for link in soup.find_all('a', href=True):
            href = link['href'].split('?')[0]
            full_url = urljoin(url, href)
            if full_url.startswith(self.base_url) and full_url not in self.visited_urls:
                self._scrape_recursive(full_url, current_depth + 1, max_depth)

    def to_json(self, filename: str):
        db_knowledge = json.loads(Path(filename).read_text())
        db_knowledge["faq"].extend(self.results)
        Path(filename).write_text(json.dumps(db_knowledge, indent=2, ensure_ascii=False))


    @tool("Agrupar texto en formato pregunta-respuesta")
    def _group_text(text: str) -> List[dict]:
        """
        Agrupa texto en una lista de objetos JSON con formato question-answer
        """
        llm = ChatOpenAI(temperature=0.1, model="gpt-4o-mini", openai_api_key=os.environ["OPENAI_API_KEY"])
        prompt = ChatPromptTemplate.from_messages([("system", "Del siguiente texto, para cada punto escribe una pregunta respondida por el texto. Luego entrega una lista de objetos JSON con atributos 'question' y 'answer'. Donde 'question' es la pregunta y 'answer' es la respuesta tomada tal como aparece en el texto. A continuación, el texto (recuerda responder con un fragmento de código markdown de un blob json con una única acción, y NADA más): \n\n {text}")])
        chain = prompt | llm
        grouped_text = chain.invoke({"text": text})
        parsed_qas = "".join(grouped_text.content.split('\n')[1:-1])
        question_answer_list = json.loads(parsed_qas)
        return question_answer_list


In [245]:
start_url = "https://www.falabella.com/falabella-cl/page/contactanos"
example_url = "https://www.falabella.com/falabella-cl/page/cambios-reemplazos"

scraper = FAQScraper(
    base_url="https://www.falabella.com/falabella-cl/page/",
    model="gpt-4o-mini",
    openai_api_key=os.environ["OPENAI_API_KEY"])

result = scraper.scrape(start_url, max_depth=2)
scraper.to_json("db_knowledge.json")



Scraping URL: https://www.falabella.com/falabella-cl/page/contactanos
Scraping URL: https://www.falabella.com/falabella-cl/page/Vende-en-Falabella.com
Scraping URL: https://www.falabella.com/falabella-cl/page/Cambios-y-Devoluciones
Scraping URL: https://www.falabella.com/falabella-cl/page/consultas-boletas-facturas
Scraping URL: https://www.falabella.com/falabella-cl/page/ganadores-concursos
Scraping URL: https://www.falabella.com/falabella-cl/page/canal-de-integridad
Scraping URL: https://www.falabella.com/falabella-cl/page/proteccion-de-datos
Scraping URL: https://www.falabella.com/falabella-cl/page/venta-empresa-falabella
Scraping URL: https://www.falabella.com/falabella-cl/page/cyber-monday
Scraping URL: https://www.falabella.com/falabella-cl/page/Black-Friday
Scraping URL: https://www.falabella.com/falabella-cl/page/cyber-day
Scraping URL: https://www.falabella.com/falabella-cl/page/falabella-retail
Scraping URL: https://www.falabella.com/falabella-cl/page/comprar-terminos-condici

FileNotFoundError: [Errno 2] No such file or directory: 'db_knowledge.json'

In [247]:
scraper.results

[{'question': '¿Qué puedo hacer en el centro de ayuda?',
  'answer': 'Revisa y gestiona tus pedidos, Sigue tus compras, Cancela tus pedidos, Haz tus solicitudes de devolución, Revisa tus boletas y tickets de cambio, Ingresa tus casos.',
  'category': 'Centro de ayuda'},
 {'question': '¿Cómo puedo contactar al centro de ayuda?',
  'answer': 'Llámanos 600 329 2002.',
  'category': 'Centro de ayuda'},
 {'question': '¿Cuáles son los horarios para hablar por teléfono?',
  'answer': 'Hablemos de lunes a sábados de 09:00 a 19:00 h.',
  'category': 'Centro de ayuda'},
 {'question': '¿Puedo hablar por WhatsApp?',
  'answer': 'Hablemos por WhatsApp.',
  'category': 'Centro de ayuda'},
 {'question': '¿Qué número debo llamar para comprar por teléfono?',
  'answer': 'Compremos por teléfono 600 390 6500.',
  'category': 'Centro de ayuda'},
 {'question': '¿Cuáles son los horarios para comprar por teléfono?',
  'answer': 'Te ayudamos a comprar de lunes a sábados de 09:00 a 22:00 h. domingos y festivos

In [250]:
scraper.to_json("../db_knowledge.json")

In [111]:
url = "https://www.falabella.com/falabella-cl/page/Vende-en-Falabella.com"
response = requests.get(url, timeout=10)  # Add a 30-second timeout
soup = BeautifulSoup(response.text, 'html.parser')

# Extract text content
text_content = soup.get_text(separator='\n', strip=True)
topic = text_content.split('\n')[0].split('|')[0].strip()

# Use LangChain agent to group text
relevant_text = scraper.llm.invoke(f"Extrae del siguiente texto el contenido relacionado con el siguiente tema: {topic}, solamente entrega el texto  \n\n{text_content}").content

In [38]:
scraper._group_text.args_schema.schema()

{'description': 'Agrupa texto en un JSON con formato question-answer',
 'properties': {'self': {'title': 'Self'},
  'text': {'description': 'Texto a agrupar',
   'title': 'Text',
   'type': 'string'}},
 'required': ['self', 'text'],
 'title': 'Agrupar texto en formato pregunta-respuesta',
 'type': 'object'}